In [1]:
import pymongo

In [2]:
###client = pymongo.MongoClient("mongodb+srv://<concealed>@cluster0.5ffcs.mongodb.net/myFirstDatabase?retryWrites=true&w=majority")
db = client.myFirstDatabase

## Q1

In [3]:
'''
pipeline=[
    {"$project":{"date":{"$concat":[{"$convert": {"input":{'$year':"$date"},"to":"string"}},"-",{"$convert": {"input":{'$month':"$date"},"to":"string"}}]},"total_price":{"$multiply":[ "$unitPrice", "$quantity" ]}}},
    {"$group":{"_id":"$date","monthlySales":{"$sum":"$total_price"}}},
    {"$sort":{"monthlySales":1}}
]
'''

'\npipeline=[\n    {"$project":{"date":{"$concat":[{"$convert": {"input":{\'$year\':"$date"},"to":"string"}},"-",{"$convert": {"input":{\'$month\':"$date"},"to":"string"}}]},"total_price":{"$multiply":[ "$unitPrice", "$quantity" ]}}},\n    {"$group":{"_id":"$date","monthlySales":{"$sum":"$total_price"}}},\n    {"$sort":{"monthlySales":1}}\n]\n'

In [4]:
pipeline=[
    {"$project":{"time":{"$substr":[{"$convert": {"input":'$date',"to":"string"}},0,7]},"total_price":{"$multiply":[ "$unitPrice", "$quantity" ]}}},
    {"$group":{"_id":"$time","monthlySales":{"$sum":"$total_price"}}},
    {"$sort":{"monthlySales":1}}
]

In [17]:
for i in db['widgetSales'].aggregate(pipeline):
    db["widgetSalesMonthlyAgg"].insert_one(i)

In [5]:
vals=db["widgetSalesMonthlyAgg"].find()
for val in vals:
    print(val)

{'_id': '2019-01', 'monthlySales': Decimal128('3496')}
{'_id': '2018-12', 'monthlySales': Decimal128('4366')}


## Q2

In [10]:
pipeline=[
    {"$match":{"productName": "Steel beam"}},
    { "$project" : { "_id":0,"productName" : 1 , "status" : 1 } } 
]


In [11]:
for i in db['orders'].aggregate(pipeline):
    print(i)

{'productName': 'Steel beam', 'status': 'new'}
{'productName': 'Steel beam', 'status': 'urgent'}
{'productName': 'Steel beam', 'status': 'urgent'}


In [21]:
db['orders'].create_index([('productName',1)])
db['orders'].create_index([('status',1)])

'status_1'

In [22]:
db.command('collStats', 'orders') 

{'ns': 'myFirstDatabase.orders',
 'size': 438,
 'count': 6,
 'avgObjSize': 73,
 'storageSize': 20480,
 'freeStorageSize': 0,
 'capped': False,
 'nindexes': 3,
 'indexDetails': {'_id_': {'metadata': {'formatVersion': 8},
   'creationString': 'access_pattern_hint=none,allocation_size=4KB,app_metadata=(formatVersion=8),assert=(commit_timestamp=none,durable_timestamp=none,read_timestamp=none,write_timestamp=off),block_allocation=best,block_compressor=,cache_resident=false,checksum=on,colgroups=,collator=,columns=,dictionary=0,encryption=(keyid=,name=),exclusive=false,extractor=,format=btree,huffman_key=,huffman_value=,ignore_in_memory_cache_size=false,immutable=false,import=(enabled=false,file_metadata=,repair=false),internal_item_max=0,internal_key_max=0,internal_key_truncate=true,internal_page_max=16k,key_format=u,key_gap=10,leaf_item_max=0,leaf_key_max=0,leaf_page_max=16k,leaf_value_max=0,log=(enabled=false),lsm=(auto_throttle=true,bloom=true,bloom_bit_count=16,bloom_config=,bloom_hash_

We can see the size of index "productName_1" is 20480 and the size of 'status_1' is also 20480. It will be the fastest because it makes our query the covered query and we always want to filter by productName. The index let us do it without go through all the records.